In [5]:
#!/usr/bin/env python3
# -*- coding: latin-1 -*-
"""
Created on Mon Aug 14 22:14:55 2017
@author: rodrigo
"""

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import re

In [6]:
# Importing the dataset
# by a first class
def get_dataset(first_class = 'H', max_N=0):
    
    id = 0
    classes = {}
    X = []
    y = []
    
    with open('../GitHub/output/full.csv', 'r', encoding="latin-1") as file:
        i_N = 0
        for line in file:
            if max_N == 0 or i_N < max_N:
                data = line.split('|')
                if len(data) == 7:
                    data_classes = data[1].split(',')
                    cl_set = set()
                    for cl in data_classes:
                        c = cl.upper()
                        c = c.replace(' ', '')
                        a = re.search(r'([A-H])([0-9]{2})([A-Z])', c, re.M|re.I)
                        try:
                            if a.group(1) == first_class:
                                cl_set.add(id)
                                if (a.group(1) + a.group(2) + a.group(3)) not in classes:
                                    classes[a.group(1) + a.group(2) + a.group(3)] = id
                                    id += 1 
                        except:
                            pass
                    if len(cl_set) > 0:
                        X.append([data[2], data[3]])
                        y.append(list(cl_set))
                        i_N += 1
    return [X, y, classes]


In [7]:
# index 0: title, summary
# index 1: list of classes by id
# index 2: classes by name and their respective ids
dataset = get_dataset(first_class = 'H')

In [8]:
# example
dataset[0][0:5]

[["Envolt'orio para m'aquinas el'etricas",
  "'E proposto em envolt'orio para m'aquinas el'etricas, sobretudo dispositivos de arranque de motores de combust~ao interna, o qual serve para a prote$c~ao das m'aquinas el'etricas contra a penetra$c~ao de sujidade e umidade, bem como contra corros~ao. O envolt'orio consiste de pelo menos uma folheta (7) que est'a empurrada ou puxada sobre as pe$cas (2 a 5) da m'aquina el'etrica a serem envolvidas, pode estar colada sobre as pe$cas (2 a 5) e est'a fixamente contra'ida sobre as pe$cas envoltas (2 a 5). Em compara$c~ao com os convencionais revestimentos de verniz, o envolt'orio forma uma prote$c~ao simples, mais favor'avel ambientalmente e mais apropriada para a fabrica$c~ao econ^omica em grandes quantidades."],
 ["Aperfei$coamento em interruptor de luz temporizado anal'ogico/digital",
  "A presente patente se refere a um interruptor temporizado em que 'e poss'ivel optar pelo acionamento digital ou anal'ogico devido ao mesmo ser dotado de uma t

In [9]:
# Binarize the labels
# 0 if it is not part of the class
# 1 otherwise
X = dataset[0]
y = dataset[1]

from sklearn.preprocessing import MultiLabelBinarizer
y = MultiLabelBinarizer().fit_transform(y)

In [10]:
# Cleaning the texts
# remove everything but letters
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
corpus_title = []
corpus_summary = []
stemmer = SnowballStemmer('portuguese')
for i in range(len(X)):
    title = X[i][0].split()
    summary = X[i][1].split()
    for j in range(len(title)):
        title[j] = re.sub('[^a-zA-Z]', '', title[j])
    for j in range(len(summary)):
        summary[j] = re.sub('[^a-zA-Z]', '', summary[j])
    title = [stemmer.stem(st) for st in title]
    summary = [stemmer.stem(st) for st in summary]
    title = ' '.join(title)
    summary = ' '.join(summary)
    corpus_title.append(title)
    corpus_summary.append(summary)

In [11]:
print('Size of X: '+str(len(X)))

Size of X: 36137


In [12]:
corpus_summary[0]

'e propost em envoltori par maquin eletr sobretud disposit de arranqu de motor de combusta intern o qual serv par a proteca das maquin eletr contr a penetraca de sujidad e umidad bem com contr corrosa o envoltori cons de pel men uma folhet  que esta empurr ou pux sobr as pec  a  da maquin eletr a ser envolv pod estar col sobr as pec  a  e esta fix contra sobr as pec envolt  a  em comparaca com os convencion revest de verniz o envoltori form uma proteca simpl mais favoravel ambiental e mais apropri par a fabricaca econom em grand quantidad'

In [13]:
corpus_summary[1]

'a present patent se refer a um interruptor temporiz em que e possivel optar pel acion digital ou analog dev ao mesm ser dot de uma tecl convencional e um sensor de toqu send que o segund ao ser acion mant a lamp aces com mei potenc por quatr minut cons o conjunt num espelh  e um suport  dupl ident ao convencional send no lug de uma das tecl fix uma chap metal  cont um led  de sinalizaca e atras do suport  localizas uma tamp  acondicion a plac  do circuit eletron'

In [14]:
corpus_title[0]

'envoltori par maquin eletr'

In [15]:
corpus_title[1]

'aperfeico em interruptor de luz temporiz analogicodigital'

In [16]:
# Vectorize titles and summary separately
from sklearn.feature_extraction.text import CountVectorizer
title_cv = CountVectorizer(max_features = None, ngram_range=(1,1))
X_title = title_cv.fit_transform(corpus_title)
summary_cv = CountVectorizer(max_features = None, ngram_range=(1,1))
X_summary = summary_cv.fit_transform(corpus_summary)

In [17]:
# Apply SVD transformation n_components=50 separately
from sklearn.decomposition import TruncatedSVD
svd_title = TruncatedSVD(n_components=50)
X_title = svd_title.fit_transform(X_title)

from sklearn.decomposition import TruncatedSVD
svd_summary = TruncatedSVD(n_components=50)
X_summary = svd_summary.fit_transform(X_summary)

In [18]:
# Concatenate title and summary svd representation
X = np.concatenate((X_title, X_summary), axis=1)

In [19]:
X[0]

array([  2.32604800e-01,   6.73060106e-01,  -5.88471086e-01,
         6.40892210e-02,  -2.30426279e-01,   4.85240270e-02,
         1.30155553e-01,   4.11557240e-02,   5.45376530e-02,
         2.14415312e-02,   1.15051873e-01,  -1.08855159e-01,
        -1.62674444e-01,  -3.98076888e-02,  -5.60035074e-02,
         3.26941379e-03,  -4.54804431e-02,  -4.37540331e-02,
        -2.63705175e-02,   1.47265490e-02,   1.62226405e-02,
        -3.10923604e-02,   2.24983101e-04,   5.11539553e-03,
        -1.36211923e-04,   2.08618170e-02,   1.36620175e-02,
         5.61329901e-03,  -6.46851080e-03,   2.35321138e-03,
        -4.27248959e-02,  -2.10998952e-02,  -2.36182676e-02,
         2.35304894e-03,  -4.22079094e-03,   1.74575673e-02,
        -3.13885984e-03,  -2.88541954e-02,   9.46968165e-03,
         8.19532525e-03,   9.61488392e-03,  -3.51110897e-03,
        -1.95592017e-02,   8.41152202e-03,  -2.98891634e-02,
         4.16890677e-03,  -1.24494878e-02,  -3.38033194e-02,
         2.59196568e-03,